In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


## Preprocesado de datos

##### En este apartado, en primer lugar cargaremos las primersa líbrerias y tambíen  los datos.

##### El segundo paso es visualizar los datos y gestionar los nulos de la mejor forma posible.

**TIP :** *Para que tu codigo sea más eficaz y en casa de necesitarlo realizar futuras actualizaciones es recomendable el uso de funciones*

In [2]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test= pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

Visualizamos como es el dataframe

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [5]:
def preprocesado(df):
    df=df.drop(['Alley'], axis=1) # Eliminamos la columna Alley
    df=df.dropna(subset=['MasVnrType']) # Eliminamos las filas con nulos
    df['BsmtFinType2']=df['BsmtFinType2'].fillna('Unf') # Asumimos los nulos como Unf (Sin terminar)
    df['LotFrontage']=df['LotFrontage'].fillna(df['LotFrontage'].median()) # Imputamos los nulos con la mediana
    df['BsmtQual']=df['BsmtQual'].fillna(0) # Imputamos los nulos por 0 (No tienen sotano)
    df['BsmtCond']=df['BsmtCond'].fillna(0) # Imputamos los nulos por 0 (No tienen sotano)
    df['BsmtExposure']=df['BsmtExposure'].fillna(0) # Imputamos los nulos por 0 (No tienen sotano)
    df['BsmtFinType1']=df['BsmtFinType1'].fillna(0) # Imputamos los nulos por 0 (No tienen sotano)
    df=df.dropna(subset=['Electrical']) # Eliminamos el error en Electrical
    df['FireplaceQu']=df['FireplaceQu'].fillna(0) # Imputamos los nulos por 0 (No tienen chimenea)
    df['GarageType']=df['FireplaceQu'].fillna('NonHave') # Imputamos los nulos por 'NonHave' (No tienen garaje)
    df['GarageYrBlt']=df['GarageYrBlt'].fillna(0) # Imputamos los nulos por 0 (Indicamos que los que no tienen garaje)
    df['GarageFinish']=df['GarageFinish'].fillna('NonHave') # Imputamos los nulos por 'NonHave' (No tienen garaje)
    df['GarageQual']=df['GarageQual'].fillna(0) # Indicamos que el garaje no existe
    df['GarageCond']=df['GarageCond'].fillna(0) # Indicamos que el garaje no existe
    df=df.drop(['PoolQC'], axis=1) # Eliminamos la columna Calidad de la piscina (Praticamente toda vacia)
    df['MiscFeature']=df['MiscFeature'].fillna('NonHave') # Indicamos que no tiene características extras
    df['Fence']=df['Fence'].fillna('NonHave') # Indicamos que no tiene vallas
    return df

In [6]:
train=preprocesado(train)

## Features Selection

##### En esta sección vamos a seleccionar solo las variables que creamos que tienen sentido para nuestro modelo, cada elección tiene que tener un porque.
##### Sobrecargar un modelo puede provocar largos periodos de espera y hacer que nuestro modelo no encuentre las mejores relaciones.

*Es necesario leerse el apartado Data Field de la página '[Datos](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data)' de la descripción de la competición*

In [7]:
features=['MSSubClass','MSZoning','LandContour','Utilities','LotConfig','Neighborhood','BldgType','HouseStyle','OverallQual','OverallCond','YearRemodAdd','ExterQual','ExterCond','BsmtCond','Heating','HeatingQC','CentralAir','Electrical','GrLivArea','FullBath','KitchenQual','TotRmsAbvGrd','FireplaceQu','GarageCars','GarageCond','Fence','MiscVal','SaleType','SaleCondition']
target=['SalePrice']

In [8]:
def select_features(df):
    train_df=df[features+target]
    return train_df

In [9]:
train_df=select_features(train)

In [10]:
train_df

,MSSubClass,MSZoning,LandContour,Utilities,LotConfig,Neighborhood,BldgType,HouseStyle,OverallQual,OverallCond,...,KitchenQual,TotRmsAbvGrd,FireplaceQu,GarageCars,GarageCond,Fence,MiscVal,SaleType,SaleCondition,SalePrice
0,60,RL,Lvl,AllPub,Inside,CollgCr,1Fam,2Story,7,5,...,Gd,8,0,2,TA,NonHave,0,WD,Normal,208500
1,20,RL,Lvl,AllPub,FR2,Veenker,1Fam,1Story,6,8,...,TA,6,TA,2,TA,NonHave,0,WD,Normal,181500
2,60,RL,Lvl,AllPub,Inside,CollgCr,1Fam,2Story,7,5,...,Gd,6,TA,2,TA,NonHave,0,WD,Normal,223500
3,70,RL,Lvl,AllPub,Corner,Crawfor,1Fam,2Story,7,5,...,Gd,7,Gd,3,TA,NonHave,0,WD,Abnorml,140000
4,60,RL,Lvl,AllPub,FR2,NoRidge,1Fam,2Story,8,5,...,Gd,9,TA,3,TA,NonHave,0,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,Lvl,AllPub,Inside,Gilbert,1Fam,2Story,6,5,...,TA,7,TA,2,TA,NonHave,0,WD,Normal,175000
1456,20,RL,Lvl,AllPub,Inside,NWAmes,1Fam,1Story,6,6,...,TA,7,TA,2,TA,MnPrv,0,WD,Normal,210000
1457,70,RL,Lvl,AllPub,Inside,Crawfor,1Fam,2Story,7,9,...,Gd,9,Gd,1,TA,GdPrv,2500,WD,Normal,266500
1458,20,RL,Lvl,AllPub,Inside,NAmes,1Fam,1Story,5,6,...,Gd,5,0,1,TA,NonHave,0,WD,Normal,142125


In [11]:
object_cols=train_df.select_dtypes(include=['object'])
object_cols=object_cols.columns.to_list()
object_cols

['MSZoning',
 'LandContour',
 'Utilities',
 'LotConfig',
 'Neighborhood',
 'BldgType',
 'HouseStyle',
 'ExterQual',
 'ExterCond',
 'BsmtCond',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'FireplaceQu',
 'GarageCond',
 'Fence',
 'SaleType',
 'SaleCondition']

In [12]:
train_df=pd.get_dummies(data=train_df, columns=object_cols)

In [13]:
train_df.head()

,MSSubClass,OverallQual,OverallCond,YearRemodAdd,GrLivArea,FullBath,TotRmsAbvGrd,GarageCars,MiscVal,SalePrice,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,7,5,2003,1710,2,8,2,0,208500,...,0,0,0,1,0,0,0,0,1,0
1,20,6,8,1976,1262,2,6,2,0,181500,...,0,0,0,1,0,0,0,0,1,0
2,60,7,5,2002,1786,2,6,2,0,223500,...,0,0,0,1,0,0,0,0,1,0
3,70,7,5,1970,1717,1,7,3,0,140000,...,0,0,0,1,1,0,0,0,0,0
4,60,8,5,2000,2198,2,9,3,0,250000,...,0,0,0,1,0,0,0,0,1,0


In [14]:
features=train_df.columns.tolist()
features.remove('SalePrice')

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_df[features], train_df[target], test_size=0.2, random_state=123)

## Técnicas multiples de regresión 

### Regresión lineal multiple

In [16]:
from sklearn.linear_model import LinearRegression

linearregress = LinearRegression()

linearregress.fit(X_train, y_train)

LinearRegression()

In [17]:
from sklearn.metrics import mean_absolute_error

predicted_linear = linearregress.predict(X_test)
linear_error=mean_absolute_error(y_test, predicted_linear)
linear_error

19681.74796596476

### Árbol de decisión

In [18]:
from sklearn.tree import DecisionTreeRegressor

treeregress = DecisionTreeRegressor(random_state=123)

treeregress.fit(X_train, y_train)

DecisionTreeRegressor(random_state=123)

In [19]:
from sklearn.metrics import mean_absolute_error

predicted_tree = treeregress.predict(X_test)
tree_error=mean_absolute_error(y_test, predicted_tree)
tree_error

29539.1529209622

### Random Forest

In [20]:
from sklearn.ensemble import RandomForestRegressor
randomregress = RandomForestRegressor(random_state=123)

randomregress.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


RandomForestRegressor(random_state=123)

In [21]:
predicted_random = randomregress.predict(X_test)
random_error=mean_absolute_error(y_test, predicted_random)
random_error

19252.439080756012

### XG REGRESSOR

In [22]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, RepeatedKFold
import multiprocessing

In [23]:
# Grid de hiperparámetros evaluados
# ==============================================================================
param_grid = {'max_depth'        : [3, 5, 10, 20],
              'subsample'        : [0.5, 1],
              'learning_rate'    : [0.01, 0.1],
              'booster'          : ['gbtree']
             }


# Crear conjunto de validación
# ==============================================================================
np.random.seed(123)
idx_validacion = np.random.choice(
                    X_train.shape[0],
                    size= int(X_train.shape[0]*0.1),
                    replace=False
                 )

X_val = X_train.iloc[idx_validacion, :].copy()
y_val = y_train.iloc[idx_validacion].copy()

X_train_grid = X_train.reset_index(drop = True).drop(idx_validacion, axis = 0).copy()
y_train_grid = y_train.reset_index(drop = True).drop(idx_validacion, axis = 0).copy()

# XGBoost necesita pasar los paramétros específicos del entrenamiento al llamar
# al método .fit()
fit_params = {"early_stopping_rounds" : 5, 
              "eval_metric"           : "rmse", 
              "eval_set"              : [(X_val, y_val)],
              "verbose"               : 0
             }

# Búsqueda por grid search con validación cruzada
# ==============================================================================
grid = GridSearchCV(
        estimator  = XGBRegressor(
                        n_estimators = 1000,
                        random_state = 123
                    ),
        param_grid = param_grid,
        scoring    = 'neg_mean_absolute_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=3, n_repeats=1, random_state=123), 
        refit      = True,
        verbose    = 0,
        return_train_score = True
       )

grid.fit(X = X_train, y = y_train, **fit_params)

# Resultados
# ==============================================================================
resultados = pd.DataFrame(grid.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head(4)

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  UserWarning,
/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,
/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  UserWarning,
/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,
/opt/conda/lib/p

,param_booster,param_learning_rate,param_max_depth,param_subsample,mean_test_score,std_test_score,mean_train_score,std_train_score
4,gbtree,0.01,10,0.5,-17971.485060,615.251600,-5417.034778,1803.778745
6,gbtree,0.01,20,0.5,-18100.001168,596.747473,-3368.511946,2276.046871
2,gbtree,0.01,5,0.5,-18180.219060,641.654386,-10502.390669,937.118021
9,gbtree,0.1,3,1,-18526.246532,626.158566,-8994.605419,2349.718247


In [24]:
# Mejores hiperparámetros por validación cruzada
# ==============================================================================
print("----------------------------------------")
print("Mejores hiperparámetros encontrados (cv)")
print("----------------------------------------")
print(grid.best_params_, ":", grid.best_score_, grid.scoring)

# Número de árboles del modelo final (early stopping)
# ==============================================================================
n_arboles_incluidos = len(grid.best_estimator_.get_booster().get_dump())
print(f"Número de árboles incluidos en el modelo: {n_arboles_incluidos}")

----------------------------------------
Mejores hiperparámetros encontrados (cv)
----------------------------------------
{'booster': 'gbtree', 'learning_rate': 0.01, 'max_depth': 10, 'subsample': 0.5} : -17971.485059992305 neg_mean_absolute_error
Número de árboles incluidos en el modelo: 1000


In [25]:
XGregress = grid.best_estimator_
predicted_XG = XGregress.predict(X_test)
XG_error=mean_absolute_error(y_test, predicted_XG)
XG_error

18131.464347079036

### LightGBM

*Añadimos GridSearchCV

In [26]:
from lightgbm.sklearn import LGBMRegressor


In [27]:
# Grid de hiperparámetros evaluados
# ==============================================================================
param_grid = {'n_estimators'     : [100, 500, 1000, 5000],
              'max_depth'        : [5, 10, 20],
              'subsample'        : [0.5, 1],
              'learning_rate'    : [0.01, 0.1],
              'boosting_type'    : ['gbdt']
             }

# Búsqueda por grid search con validación cruzada
# ==============================================================================
grid = GridSearchCV(
        estimator  = LGBMRegressor(random_state=123),
        param_grid = param_grid,
        scoring    = 'neg_mean_absolute_error',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=3, n_repeats=1, random_state=123), 
        refit      = True,
        verbose    = 0,
        return_train_score = True
       )

grid.fit(X = X_train, y = y_train)

# Resultados
# ==============================================================================
resultados = pd.DataFrame(grid.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head(4)

,param_boosting_type,param_learning_rate,param_max_depth,param_n_estimators,param_subsample,mean_test_score,std_test_score,mean_train_score,std_train_score
7,gbdt,0.01,5,5000,1,-18701.020815,267.090123,-8726.521264,856.286038
6,gbdt,0.01,5,5000,0.5,-18701.020815,267.090123,-8726.521264,856.286038
27,gbdt,0.1,5,500,1,-18780.327530,381.337302,-8481.486577,863.590067
26,gbdt,0.1,5,500,0.5,-18780.327530,381.337302,-8481.486577,863.590067


In [28]:
# Mejores hiperparámetros por validación cruzada
# ==============================================================================
print("----------------------------------------")
print("Mejores hiperparámetros encontrados (cv)")
print("----------------------------------------")
print(grid.best_params_, ":", grid.best_score_, grid.scoring)

----------------------------------------
Mejores hiperparámetros encontrados (cv)
----------------------------------------
{'boosting_type': 'gbdt', 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 5000, 'subsample': 0.5} : -18701.02081489661 neg_mean_absolute_error


In [29]:
Light_regress = grid.best_estimator_
predicted_Light = Light_regress.predict(X = X_test,)
Light_error=mean_absolute_error(y_test, predicted_Light)
Light_error

19465.838286595226

In [30]:
from sklearn.inspection import permutation_importance
importancia = permutation_importance(
                estimator    = Light_regress,
                X            = X_train,
                y            = y_train,
                n_repeats    = 5,
                scoring      = 'neg_root_mean_squared_error',
                n_jobs       = multiprocessing.cpu_count() - 1,
                random_state = 123
             )

# Se almacenan los resultados (media y desviación) en un dataframe
df_importancia = pd.DataFrame(
                    {k: importancia[k] for k in ['importances_mean', 'importances_std']}
                 )
df_importancia['feature'] = X_train.columns
df_importancia.sort_values('importances_mean', ascending=False)

,importances_mean,importances_std,feature
4,34325.853355,643.520914,GrLivArea
1,31141.758682,1049.846071,OverallQual
7,8177.577241,316.696482,GarageCars
3,5948.071398,355.061712,YearRemodAdd
0,4586.446414,290.696105,MSSubClass
...,...,...,...
59,0.000000,0.000000,HouseStyle_2.5Unf
100,0.000000,0.000000,FireplaceQu_Ex
70,0.000000,0.000000,ExterCond_Po
103,0.000000,0.000000,FireplaceQu_Po
